#**Importing Necessary libraries**


In [ ]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re
from keras.layers import SimpleRNN

# **Preparing the data named IMDB**

In [ ]:
data = pd.read_csv('/content/IMDB Dataset.csv')
print(data)

ParserError: Error tokenizing data. C error: EOF inside string starting at row 38010

Stop Word is a commonly used words in a sentence, usually a search engine is programmed to ignore this words (i.e. "the", "a", "an", "of", etc.)
Declaring the english stop words

In [ ]:
import nltk
nltk.download("stopwords")
english_stops = set(stopwords.words('english'))

# **Load and Clean Dataset**
**In the original dataset, the reviews are still dirty. There are still html tags, numbers, uppercase, and punctuations. This will not be good for training, so in load_dataset() function, beside loading the dataset using pandas, I also pre-process the reviews by removing html tags, non alphabet (punctuations and numbers), stop words, and lower case all of the reviews.**

# **Encode Sentiments**
**In the same function, We also encode the sentiments into integers (0 and 1). Where 0 is for negative sentiments and 1 is for positive sentiments.**

In [ ]:
def load_dataset():
    df = pd.read_csv('/content/drive/MyDrive/AMITY/Deep Learning (codes)/Data/IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/AMITY/Deep Learning (codes)/Data/IMDB Dataset.csv'

#**Split Dataset**
**In this work, We decided to split the data into 80% of Training and 20% of Testing set using train_test_split method from Scikit-Learn. By using this method, it automatically shuffles the dataset. We need to shuffle the data because in the original dataset, the reviews and sentiments are in order, where they list positive reviews first and then negative reviews. By shuffling the data, it will be distributed equally in the model, so it will be more accurate for predictions.**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
22596    [boring, badly, written, italian, exploitation...
5353     [the, other, supposed, horror, movie, made, it...
42152    [a, tough, life, gets, tougher, three, childre...
15434    [why, earth, colin, firth, pointless, film, ha...
7280     [this, far, worst, movie, i, ever, seen, cinem...
                               ...                        
39945    [this, show, lasted, moments, plots, usually, ...
13858    [i, rented, thinking, would, pretty, good, cov...
25266    [having, pleasantly, surprised, sandra, bulloc...
10659    [the, difficulty, i, musical, version, les, mi...
39372    [this, movie, proof, film, noire, enduring, st...
Name: review, Length: 40000, dtype: object 

2006     [this, movie, time, favorite, you, really, see...
33575    [this, british, film, version, stage, play, i,...
6808     [alexander, nevsky, brilliant, piece, cinemati...
32330    [found, old, vhs, version, film, parents, hous...
3777     [i, went, see, movie, daughter, i, insisted, g...
 

**Function for getting the average review length, by calculating the mean of all the reviews length (using numpy.mean)**

In [ ]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

#**Tokenize and Pad/Truncate Reviews**
**A Neural Network only accepts numeric data, so we need to encode the reviews. I use tensorflow.keras.preprocessing.text.Tokenizer to encode the reviews into integers, where each unique word is automatically indexed (using fit_on_texts method) based on x_train.**

**x_train and x_test is converted into integers using texts_to_sequences method.**

**Each reviews has a different length, so we need to add padding (by adding 0) or truncating the words to the same length (in this case, it is the mean of all reviews length) using tensorflow.keras.preprocessing.sequence.pad_sequences.**

In [ ]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding
print('Total Words:', total_words)

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Total Words: 92636
Encoded X Train
 [[  257   863   310 ...     0     0     0]
 [    2  1340   350 ...    28   282   409]
 [   39  1138    40 ...     0     0     0]
 ...
 [ 1587  3903   660 ...    62 14457  1006]
 [    2  6090     1 ...  4973  5675   406]
 [    8     3  2912 ...     0     0     0]] 

Encoded X Test
 [[    8     3    10 ...     0     0     0]
 [    8   603     4 ...   278 10278  2289]
 [ 3551 11276   417 ...     0     0     0]
 ...
 [ 3908 20405  3718 ...     0     0     0]
 [   39  2984   218 ...  3947     3   765]
 [    1   260  1833 ...     0     0     0]] 

Maximum review length:  130


#**Build Architecture/Model**
**Embedding Layer: in simple terms, it creates word vectors of each word in the word_index and group words that are related or have similar meaning by analyzing other words around them.**

**RNN Layer: to make a decision to keep or throw away data by considering the current input, previous output.**

**Dense Layer: compute the input with the weight matrix and bias (optional), and using an activation function. I use Sigmoid activation function for this work because the output is only 0 or 1.**

**The optimizer is Adam and the loss function is Binary Crossentropy because again the output is only 0 and 1, which is a binary number.**

In [ ]:
rnn = Sequential()

rnn.add(Embedding(total_words,32,input_length =max_length))
rnn.add(SimpleRNN(64,input_shape = (total_words, max_length), return_sequences=False,activation="relu"))
rnn.add(Dense(1, activation = 'sigmoid')) #flatten

print(rnn.summary())
rnn.compile(loss="binary_crossentropy",optimizer='adam',metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 130, 32)           2964352   
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                6208      
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2,970,625
Trainable params: 2,970,625
Non-trainable params: 0
_________________________________________________________________
None


#**Trainin the Model**

In [ ]:
history = rnn.fit(x_train,y_train,epochs = 20,batch_size=128,verbose = 1)

Epoch 1/20
313/313 [==============================] - 96s 286ms/step - loss: 0.6915 - accuracy: 0.5184
Epoch 2/20
313/313 [==============================] - 68s 218ms/step - loss: 0.6616 - accuracy: 0.5879
Epoch 3/20
313/313 [==============================] - 63s 202ms/step - loss: 0.6626 - accuracy: 0.5762
Epoch 4/20
313/313 [==============================] - 56s 180ms/step - loss: 0.5900 - accuracy: 0.6328
Epoch 5/20
313/313 [==============================] - 51s 162ms/step - loss: 0.4166 - accuracy: 0.8135
Epoch 6/20
313/313 [==============================] - 51s 161ms/step - loss: 0.3511 - accuracy: 0.8806
Epoch 7/20
313/313 [==============================] - 47s 149ms/step - loss: 0.2362 - accuracy: 0.9194
Epoch 8/20
313/313 [==============================] - 46s 148ms/step - loss: 0.1676 - accuracy: 0.9421
Epoch 9/20
313/313 [==============================] - 45s 144ms/step - loss: 0.3168 - accuracy: 0.8689
Epoch 10/20
313/313 [==============================] - 46s 148ms/step - l

#**Saving The Model**

In [ ]:
model = rnn.save('rnn.h5')
loaded_model = load_model('rnn.h5')

#**Evaluation**

In [ ]:
y_pred = rnn.predict(x_test, batch_size = 128)
print(y_pred)
print(y_test)
for i in range(len(y_pred)):
  if y_pred[i]>0.5:
    y_pred[i] = 1
  else:
    y_pred[i] = 0

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

79/79 [==============================] - 1s 12ms/step
[[0.78446704]
 [0.02569966]
 [0.78301245]
 ...
 [0.2700789 ]
 [0.72713566]
 [0.78446704]]
2006     1
33575    1
6808     1
32330    0
3777     0
        ..
40255    1
5864     1
44604    0
42481    1
31671    1
Name: sentiment, Length: 10000, dtype: int64
Correct Prediction: 6918
Wrong Prediction: 3082
Accuracy: 69.17999999999999


Message: **Nothing was typical about this. Everything was beautifully done in this movie, the story, the flow, the scenario, everything. I highly recommend it for mystery lovers, for anyone who wants to watch a good movie!**

#**Example review**

In [ ]:
review = str(input('Movie Review: '))

Movie Review: Nothing was typical about this. Everything was beautifully done in this movie, the story, the flow, the scenario, everything. I highly recommend it for mystery lovers, for anyone who wants to watch a good movie!


#**Pre-processing of entered review**

In [ ]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  Nothing was typical about this Everything was beautifully done in this movie the story the flow the scenario everything I highly recommend it for mystery lovers for anyone who wants to watch a good movie
Filtered:  ['nothing typical everything beautifully done movie story flow scenario everything i highly recommend mystery lovers anyone wants watch good movie']


In [ ]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[  76  705  174 1210  126    3   13 2692 2596  174    1  442  280  701
  1771  155  400   33    9    3    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]]


#**Prediction**

In [ ]:
result = rnn.predict(tokenize_words)
print(result)

1/1 [==============================] - 0s 40ms/step
[[0.78446704]]


In [ ]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive
